## Data import

We have the tick-by-tick data for 15 currency pairs, given below

    pairs = ['AUDJPY', 'AUDNZD', 'AUDUSD', 'CADJPY', 'CHFJPY', 'EURCHF', 'EURGBP', 'EURJPY',
    'EURUSD', 'GBPJPY', 'GBPUSD', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDJPY']

We download the data for the currency pairs from [TrueFX](https://truefx.com) using the `fetch` script to generate URLs and piping the output to a script that downloads the files from the URLs.

## Data structure

The CSVs downloaded have the following column structure.

    columns = ['name', 'timestamp', 'bid', 'ask']

 * _name_: The name of the currency pair
 * _timestamp_: Timestamp of the tick
 * _bid_: The price at which you can sell in the market
 * _ask_: The price at which you can buy in the market

In [14]:
import pandas as pd
import numpy as np

data_df = pd.read_csv('./AUDJPY-2012-01.csv', header=None)

data_df.columns = ['name', 'tick', 'bid', 'ask']

print(data_df.head())

      name                   tick     bid     ask
0  AUD/JPY  20120102 00:00:00.309  78.714  78.814
1  AUD/JPY  20120102 00:00:00.311  78.716  78.812
2  AUD/JPY  20120102 00:00:05.006  78.716  78.813
3  AUD/JPY  20120102 00:00:05.006  78.721  78.811
4  AUD/JPY  20120102 00:00:05.007  78.726  78.811


## Data preparation and feature extraction

1. Resampling: Resample the data into 15-minute intervals.
2. Add features: OHLCV (Open, High, Low, Close, Volume) for the intervals.


The prepared data has columns:

    columns = ['tick_start', 'tick_end', 'open', 'high', 'low', 'close', 'volume']

In [15]:
# Let the dataframe know the timestamp column

data_df['tick'] = pd.to_datetime(data_df['tick'])

In [23]:
print(data_df.head())

      name                    tick     bid     ask
0  AUD/JPY 2012-01-02 00:00:00.309  78.714  78.814
1  AUD/JPY 2012-01-02 00:00:00.311  78.716  78.812
2  AUD/JPY 2012-01-02 00:00:05.006  78.716  78.813
3  AUD/JPY 2012-01-02 00:00:05.006  78.721  78.811
4  AUD/JPY 2012-01-02 00:00:05.007  78.726  78.811


In [26]:
# Drop unnecesarry column
del data_df['name']

In [36]:
last_time = data_df['tick'][0].timestamp()
cur_price = data_df['ask'][0]

# Each 15 minute interval is represented by the following variables.
cur_interval = {'tick_start': last_time,
                'tick_end': last_time,
                'open': cur_price,
                'high': cur_price,
                'low': cur_price,
                'close': cur_price,
                'volume': 0}
intervals = []

# Sample the intervals from the raw data.
for row in data_df.itertuples():
    cur_time = row.tick.timestamp()
    cur_price = row.ask
    if cur_time - last_time > 900:
        # Append this to the list of intervals.
        intervals.append(cur_interval.copy())
        
        cur_interval = {'tick_start':cur_time,
                'tick_end': cur_time,
                'open': cur_price,
                'high': cur_price,
                'low': cur_price,
                'close': cur_price,
                'volume': 1}
        last_time = cur_time
    else:
        cur_interval['tick_end'] = cur_time
        cur_interval['close'] = cur_price
        cur_interval['high'] = max(cur_price, cur_interval['high'])
        cur_interval['low'] = min(cur_price, cur_interval['low'])
        cur_interval['volume'] += 1

        
# The last remaining (incomplete) < 15-minute interval.
intervals.append(cur_interval.copy())

# Create a dataframe.
processed_data = pd.DataFrame(intervals)

print(processed_data.head())
print(processed_data.describe())

processed_data.to_csv('AUDJPY-2012-01_features.csv')

    close    high     low    open      tick_end    tick_start  volume
0  78.823  78.847  78.808  78.814  1.325463e+09  1.325462e+09     131
1  78.824  78.836  78.761  78.822  1.325464e+09  1.325463e+09     138
2  78.855  78.867  78.820  78.820  1.325465e+09  1.325464e+09      58
3  78.844  78.857  78.803  78.857  1.325466e+09  1.325465e+09     535
4  78.810  78.846  78.765  78.846  1.325467e+09  1.325466e+09     494
             close         high          low         open      tick_end  \
count  1971.000000  1971.000000  1971.000000  1971.000000  1.971000e+03   
mean     80.019732    80.059914    79.977959    80.018264  1.326681e+09   
std       1.169100     1.170952     1.166206     1.169085  7.613004e+05   
min      78.165000    78.166000    78.101000    78.165000  1.325463e+09   
25%      79.155000    79.193000    79.121000    79.156000  1.326080e+09   
50%      79.507000    79.560000    79.465000    79.505000  1.326714e+09   
75%      80.915500    80.971500    80.878500    80.9130